In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython.display import display
pd.set_option('display.max_columns', None)

import pyspark.sql.functions as F

import os
import getpass

In [3]:
# Get clone the repo
try:
    std_out = subprocess.run(["git", "clone", "https://github.com/CSBP-CPSE/ais-data.git"],
                             capture_output=True,
                             text=True)
    print(std_out)
    
    # Create Data folder if it doesn't exist
    if os.path.exists("./ais-data/Data/") is False:
        os.mkdir("./ais-data/Data/")
    
    print(os.listdir("./ais-data/"))
except:
    print("Cloning repo failed")

CompletedProcess(args=['git', 'clone', 'https://github.com/CSBP-CPSE/ais-data.git'], returncode=0, stdout='', stderr="Cloning into 'ais-data'...\n")
['.git', '.ipynb_checkpoints', 'ODI_Marine_Ports_v0.1.csv', 'README.md', 'ais_analysis.ipynb', 'export_data.ipynb', 'move_data.ipynb', 'vancouver_port.ipynb', 'Data']


In [4]:
basepath = "s3a://ungp-ais-data-historical-backup/register/"
basepath_archive = "s3a://ungp-ais-data-historical-backup/register-archive/"

,PortName,Latitude,Longitude
0,Port of Vancouver,49.2854,-123.0805
1,Port of Saint John,45.4796,-66.0628
2,Port of Quebec,46.8210,-71.2040
3,Port of Montreal,45.5470,-73.5300
4,Come-by-Chance,47.7985,-54.0186
5,Sept-Iles,50.2133,-66.3758
6,Strait of Canso Port,45.3363,-60.9944
7,Prince Rupert,54.2883,-130.3562
8,Port-Cartier,50.0326,-66.8908
9,Offshore (St. John's),53.1355,-57.6604


380


In [5]:
# Get current version
df = spark.read.load(basepath+"version.csv", 
                     format="csv", sep=",", inferSchema="true")
df.show()

# Get the first element
print(df.collect()[0][0])

+------------------+------------+---------+-------------------+------------------+------------------+-------+-------------------+--------+-----------+----------------+-----------------+------------+------+-----+------------+---------+-----------+-------+-------+---+-----+---+-------+--------------------+---------------+------+-------------------+-------------------+----------------+---------------+----+--------------------+---------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|    H3_int_index_8|message_type|     mmsi|      dt_insert_utc|         longitude|          latitude|    imo|        vessel_name|callsign|vessel_type|vessel_type_code|vessel_type_cargo|vessel_class|length|width|flag_country|flag_code|destination|    eta|draught|sog|  cog

In [ ]:
ship_data = spark.read.load(basepath+ "ShipData.CSV", 
                     format="csv", sep=",", inferSchema="true", header="true")
ship_data = ship_data.toPandas()

# Save
ship_data.to_csv("./ais-data/Data/ship_data_{}.csv".format(df.collect()[0][0]), index=False)

In [ ]:
# Get all previous versions too
df = spark.read.load(basepath_archive+"versions.csv", 
                     format="csv", sep=",", inferSchema="true")
df = df.toPandas()

df

In [ ]:
df.dtypes

In [ ]:
df['date'] = df['_c0'].str[:4] + "/" + df['_c0']

In [ ]:
for i in range(len(df.index)):
    
    basepath_archive_version = basepath_archive + df['date'][i]
    ship_data = spark.read.load(basepath_archive_version+"/ShipData.CSV", 
                         format="csv", sep=",", inferSchema="true", header="true")
    ship_data = ship_data.toPandas()
    ship_data.to_csv("./ais-data/Data/ship_data_{}.csv".format(df['_c0'][i]), index=False)

In [10]:
# Change directory
os.chdir("./ais-data")
print(os.listdir("./Data/"))

['top20_ports_2019-03-01_to_2019-03-31_chunk_0.csv', 'top20_ports_2019-03-01_to_2019-03-31_chunk_1.csv', 'top20_ports_2019-03-01_to_2019-03-31_chunk_2.csv', 'top20_ports_2019-03-01_to_2019-03-31_chunk_3.csv', 'top20_ports_2019-03-01_to_2019-03-31_chunk_4.csv', 'top20_ports_2019-03-01_to_2019-03-31_chunk_5.csv', 'top20_ports_2019-03-01_to_2019-03-31_chunk_6.csv', 'top20_ports_2019-03-01_to_2019-03-31_chunk_7.csv', 'top20_ports_2019-03-01_to_2019-03-31_chunk_8.csv', 'top20_ports_2019-03-01_to_2019-03-31_chunk_9.csv', 'top20_ports_2019-03-01_to_2019-03-31_chunk_10.csv', 'top20_ports_2019-03-01_to_2019-03-31_chunk_11.csv', 'top20_ports_2019-03-01_to_2019-03-31_chunk_12.csv', 'top20_ports_2019-03-01_to_2019-03-31_chunk_13.csv', 'top20_ports_2019-03-01_to_2019-03-31_chunk_14.csv', 'top20_ports_2019-03-01_to_2019-03-31_chunk_15.csv', 'top20_ports_2019-03-01_to_2019-03-31_chunk_16.csv', 'top20_ports_2019-03-01_to_2019-03-31_chunk_17.csv', 'top20_ports_2019-03-01_to_2019-03-31_chunk_18.csv']


In [11]:
# Git Config
std_out = subprocess.run(["git", "config", "--global", "user.email", '"dennishuynh3@gmail.com"'], capture_output=True, text=True)
print(std_out)
std_out = subprocess.run(["git", "config", "--global", "user.name", '"DennisH3"'], capture_output=True, text=True)
print(std_out)

# Git add and commit
std_out = subprocess.run(["git", "add", "."], capture_output=True, text=True)
print(std_out)
std_out = subprocess.run(["git", "commit", "-m", "Upload data"], capture_output=True, text=True).stdout
print(std_out)

CompletedProcess(args=['git', 'config', '--global', 'user.email', '"dennishuynh3@gmail.com"'], returncode=0, stdout='', stderr='')
CompletedProcess(args=['git', 'config', '--global', 'user.name', '"DennisH3"'], returncode=0, stdout='', stderr='')
CompletedProcess(args=['git', 'add', '.'], returncode=0, stdout='', stderr='')
[main 64df685] Upload data
 19 files changed, 1803226 insertions(+)
 create mode 100644 Data/top20_ports_2019-03-01_to_2019-03-31_chunk_0.csv
 create mode 100644 Data/top20_ports_2019-03-01_to_2019-03-31_chunk_1.csv
 create mode 100644 Data/top20_ports_2019-03-01_to_2019-03-31_chunk_10.csv
 create mode 100644 Data/top20_ports_2019-03-01_to_2019-03-31_chunk_11.csv
 create mode 100644 Data/top20_ports_2019-03-01_to_2019-03-31_chunk_12.csv
 create mode 100644 Data/top20_ports_2019-03-01_to_2019-03-31_chunk_13.csv
 create mode 100644 Data/top20_ports_2019-03-01_to_2019-03-31_chunk_14.csv
 create mode 100644 Data/top20_ports_2019-03-01_to_2019-03-31_chunk_15.csv
 create 

In [12]:
# Get credentials
un = "DennisH3"
pw = getpass.getpass(prompt="Please enter PAT")

Please enter PAT········


In [13]:
push = "git push --repo https://{}:{}@github.com/CSBP-CPSE/ais-data.git".format(un, pw)

std_out = subprocess.run([push], capture_output=True, text=True, shell=True).stderr
print(std_out)

remote: warning: See http://git.io/iEPt8g for more information.        
remote: warning: File Data/top20_ports_2019-03-01_to_2019-03-31_chunk_11.csv is 61.31 MB; this is larger than GitHub's recommended maximum file size of 50.00 MB        
remote: warning: File Data/top20_ports_2019-03-01_to_2019-03-31_chunk_14.csv is 61.35 MB; this is larger than GitHub's recommended maximum file size of 50.00 MB        
remote: warning: File Data/top20_ports_2019-03-01_to_2019-03-31_chunk_8.csv is 61.26 MB; this is larger than GitHub's recommended maximum file size of 50.00 MB        
remote: warning: File Data/top20_ports_2019-03-01_to_2019-03-31_chunk_6.csv is 61.15 MB; this is larger than GitHub's recommended maximum file size of 50.00 MB        
remote: warning: File Data/top20_ports_2019-03-01_to_2019-03-31_chunk_3.csv is 61.26 MB; this is larger than GitHub's recommended maximum file size of 50.00 MB        
remote: warning: File Data/top20_ports_2019-03-01_to_2019-03-31_chunk_4.csv is 61.17 M